# Chapter 5 - Recognizing music genres with TensorFlow and the Raspberry Pi Pico
## Part 1: Data collection and MFCCs computation

## Python libraries

In [ ]:
!pip install numpy==1.23.5
!pip install cmsisdsp==1.9.6

## Import libraries

In [ ]:
import soundfile as sf
import librosa
import numpy as np
import tensorflow as tf
import cmsisdsp as dsp
import math

from google.colab import drive

## Constants

In [ ]:
# Google drive top-level directory
G_DRIVE_ROOT = '/content/drive/'

# Audio sample rate
SAMPLE_RATE = 22050

# MFFC constants
FRAME_LENGTH = 2048
FRAME_STEP = 1024
FFT_LENGTH = 2048
FMIN_HZ = 20
FMAX_HZ = SAMPLE_RATE / 2
NUM_MEL_FREQS = 40
NUM_MFCCS = 18

## Extracting MFCCs from audio samples with TensorFlow


### Implement a function to compute MFCCs with the TensorFlow signal processing functions

In [ ]:
def extract_mfccs_tf(
  ad_src,
  ad_sample_rate,
  num_mfccs,
  frame_length,
  frame_step,
  fft_length,
  fmin_hz,
  fmax_hz,
  num_mel_freqs):

  n = ad_src.shape[0]
  num_frames = int(((n - frame_length) / frame_step) + 1)

  output = np.zeros(shape=(num_frames, num_mfccs))

  # Iterate over each frame to get the MFCC coefficients
  for i in range(num_frames):
    idx_s = i * frame_step
    idx_e = idx_s + frame_length
    src = ad_src[idx_s:idx_e]

    # Apply the Hann Window in-place
    hann_coef = tf.signal.hann_window(frame_length)
    hann = src * hann_coef

    # Apply the RFFT
    fft_spect = tf.signal.rfft(hann)

    # Calculate the magnitude of the FFT
    fft_mag_spect = tf.math.abs(fft_spect)

    # Calculate the coefficients of Mel-weights for converting the spectrum from Hz to Mel
    num_fft_freqs = fft_mag_spect.shape[0]
    mel_wei_mtx = tf.signal.linear_to_mel_weight_matrix(
      num_mel_freqs,
      num_fft_freqs,
      ad_sample_rate,
      fmin_hz,
      fmax_hz)

    # Convert the spectrum to Mel
    mel_spect = np.matmul(fft_mag_spect, mel_wei_mtx)

    # Perform the log function
    log_mel_spect = np.log(mel_spect + 1e-6)

    dct = tf.signal.mfccs_from_log_mel_spectrograms(
    log_mel_spect)

    # Extract the MFFC coefficients
    output[i] = dct[0:num_mfccs]

  return output

### Mount top-level Google Drive

In [ ]:
drive.mount(G_DRIVE_ROOT)

In [ ]:
# CHANGE TO DESIRED TRAIN FOLDER
train_dir = "drive/MyDrive/mgr_dataset"

### Load an audio file

In [ ]:
filepath = train_dir + "/disco/disco.00002.wav"
ad, sr = sf.read(filepath)

print('Sample rate:', sr)
print('Sample shape:', ad.shape)
print('Song duration:', ad.shape[0] / sr)

### Play one-second of audio

In [ ]:
# Take one second of audio
test_ad = ad[0:SAMPLE_RATE]

import IPython.display as ipd
ipd.Audio(test_ad, rate=sr)

### Extract the MFCCs using TensorFlow

In [ ]:
mfccs_tf = extract_mfccs_tf(
    test_ad,
    SAMPLE_RATE,
    NUM_MFCCS,
    FRAME_LENGTH,
    FRAME_STEP,
    FFT_LENGTH,
    FMIN_HZ,
    FMAX_HZ,
    NUM_MEL_FREQS)

### Implement a function to visualize the MFCCs as image

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm

def display_mfccs(mfcc_src):
  fig, ax = plt.subplots()
  cax = ax.imshow(mfcc_src, interpolation='nearest', cmap=cm.gray, origin='lower')
  ax.set_title('MFCCs')
  plt.xlabel('Frame index - Time')
  plt.ylabel('Coefficient index - Frequency')
  plt.colorbar(cax)
  plt.show()

### Display the MFCCs

In [ ]:
display_mfccs(mfccs_tf.T)

### Implement a function to compute MFCCs algorithm with the Librosa library

In [ ]:
def extract_mfccs_librosa(
  ad_src,
  ad_sample_rate,
  num_mfccs,
  frame_length,
  frame_step,
  fft_length,
  fmin_hz,
  fmax_hz,
  num_mel_freqs):

  return librosa.feature.mfcc(
      y=ad_src,
      sr=ad_sample_rate,
      n_mfcc = num_mfccs,
      n_fft = fft_length,
      hop_length = frame_step,
      win_length = frame_length,
      center = False,
      n_mels = num_mel_freqs,
      fmin = fmin_hz,
      fmax = fmax_hz)

### Extract the MFCCs using the Librosa library


In [ ]:
mfccs_librosa = extract_mfccs_librosa(
    test_ad,
    SAMPLE_RATE,
    NUM_MFCCS,
    FRAME_LENGTH,
    FRAME_STEP,
    FFT_LENGTH,
    FMIN_HZ,
    FMAX_HZ,
    NUM_MEL_FREQS)

### Display the MFCCs

In [ ]:
display_mfccs(mfccs_librosa)

## Part 2: Model deployment

## Constants

In [ ]:
# Music genres
LIST_GENRES = ['disco', 'jazz', 'metal']

# Training audio length in seconds
TRAIN_AUDIO_LENGTH_SEC = 1

# Training audio length in number of samples
TRAIN_AUDIO_LENGTH_SAMPLES = SAMPLE_RATE * TRAIN_AUDIO_LENGTH_SEC

# TensorFlow model name
TF_MODEL = 'music_genre'

# TensorFlow lite model name
TFL_MODEL_FILE = 'music_genre.tflite'

## Computing the FFT magnitude with fixed-point arithmetic through the CMSIS-DSP Python library

### Implement a function that computes the RFFT in Q15 fixed-point using CMSIS-DSP.

In [ ]:
def rfft_q15(src):
  src_len = src.shape[0]
  inst = dsp.arm_rfft_instance_q15()
  stat = dsp.arm_rfft_init_q15(inst, src_len, 0, 1)
  fft_q = dsp.arm_rfft_q15(inst, src)
  return fft_q[:src_len + 1]

### Implement a function that computes the magnitude in Q15 fixed-point using CMSIS-DSP

In [ ]:
def mag_q15(src):
  f0 = src[0],
  fn = src[1],
  fx = dsp.arm_cmplx_mag_q15(src[2:])
  return np.concatenate((f0, fx, fn))

### Get a frame from the test audio sample

In [ ]:
src = test_ad[0:FRAME_LENGTH]

### Compute the FFT magnitude using the 16-bit fixed-point arithmetic

In [ ]:
# Convert to Q15
src_q15 = dsp.arm_float_to_q15(src)

# Apply the RFFT. The output is Q12.4. Therefore, fewer fractional bits
cmsis_fft_q15 = rfft_q15(src_q15)

# Calculate the magnitude of the FFT. The output is Q13.3
cmsis_fft_mag_q15 = mag_q15(cmsis_fft_q15)

# Convert to float
scale = float(1 << 3) # 8

cmsis_fft_mag = cmsis_fft_mag_q15 / scale

### Compute the FFT magnitude using the floating-point arithmetic

In [ ]:
tf_fft = tf.signal.rfft(src)
tf_fft_mag = tf.math.abs(tf_fft)

### Evaluate the difference stats

In [ ]:
abs_diff = np.abs(tf_fft_mag - cmsis_fft_mag)
print("Differences:\nmin:", np.min(abs_diff),
      "max:", np.max(abs_diff),
      "mean:", np.mean(abs_diff),
      "std:", np.std(abs_diff))

## Implementing the MFCCs feature extraction with the CMSIS-DSP library


### Implement a function to precompute the Hann window coefficients in Q15 format

In [ ]:
def gen_hann_lut_q15(frame_len):
  hann_lut_f32 = tf.signal.hann_window(frame_len)
  return dsp.arm_float_to_q15(hann_lut_f32)

### Implement a function to precompute the Mel-weight matrix in Q15 format

In [ ]:
def gen_mel_weight_mtx(sr, fmin_hz, fmax_hz, num_mel_freqs, num_fft_freqs):
  m_f32 = tf.signal.linear_to_mel_weight_matrix(
    num_mel_freqs,
    num_fft_freqs,
    sr,
    fmin_hz,
    fmax_hz)

  m_q15 = dsp.arm_float_to_q15(m_f32)
  # Reshape is needed because the conversion from float to q15 collapses the dimensions
  return m_q15.reshape((m_f32.shape[0], m_f32.shape[1]))

### Implement a function to precompute the logarithmic function with input as 16-bit fixed-point

In [ ]:
def gen_log_lut_q(q_scale):
  max_int16 = np.iinfo("int16").max

  log_lut = np.zeros(shape=(max_int16), dtype="int16")

  for i16 in range(0, max_int16):
    q16 = np.array([i16,], dtype="int16")
    f_v = q16 / float(q_scale)
    log_f = np.array(np.log(f_v + 1e-6),)
    log_q = log_f * float(q_scale)
    log_lut[i16] = int(log_q)

  return log_lut

### Implement a function to precompute the DCT-weight matrix in Q15 format

In [ ]:
def gen_dct_weight_mtx(num_mel_freqs, num_mfccs):
  mtx_q15 = np.zeros(shape=(num_mel_freqs, num_mfccs), dtype="int16")

  scale = np.sqrt(2.0 / float(num_mel_freqs))
  pi_div_mel = (math.pi / num_mel_freqs)
  for n in range(num_mel_freqs):
    for k in range(num_mfccs):
      v = scale * np.cos(pi_div_mel * (n + 0.5) * k)
      v_f32 = np.array([v,], dtype="float32")
      mtx_q15[n][k] = dsp.arm_float_to_q15(v_f32)
  return mtx_q15

### Precompute the Hann window coefficients, Mel-weight matrix, DCT-weight matrix, and logarithmic function

In [ ]:
# Generate the Hann Window LUT for Q15
hann_lut_q15 = gen_hann_lut_q15(FRAME_LENGTH)

# Precalculate the Mel-weight matrix in Q15 fixed-point format
mel_wei_mtx_q15 = gen_mel_weight_mtx(SAMPLE_RATE, FMIN_HZ, FMAX_HZ, NUM_MEL_FREQS, int((FFT_LENGTH / 2) + 1))

# Generate the Log LUT for Q13.3 fixed-point format
log_lut_q13_3 = gen_log_lut_q(8)

# Precalculate the DCT-II-weight matrix
dct_wei_mtx_q15 = gen_dct_weight_mtx(NUM_MEL_FREQS, NUM_MFCCS)

### Show the program memory usage

In [ ]:
mem_usage = 0
mem_usage += np.size(hann_lut_q15) * 2
mem_usage += np.size(mel_wei_mtx_q15) * 2
mem_usage += np.size(log_lut_q13_3) * 2
mem_usage += np.size(dct_wei_mtx_q15) * 2

print("Program memory usage: ", mem_usage, "bytes")

### Implement a function to compute the MFCCs feature extraction with 16-bit fixed-point arithmetic

In [ ]:
def extract_mfccs_cmsis(
  ad_src,
  num_mfccs,
  frame_length,
  frame_step,
  hann_lut_q15,
  mel_wei_mtx_q15,
  log_lut_q13_3,
  dct_wei_mtx_q15):

  n = ad_src.shape[0]
  num_frames = int(((n - frame_length) / frame_step) + 1)

  output = np.zeros(shape=(num_frames, num_mfccs))

  # Iterate over each frame to get the MFCC coefficients
  for i in range(num_frames):
    idx_s = i * frame_step
    idx_e = idx_s + frame_length
    frame = ad_src[idx_s:idx_e]

    # Convert to Q15
    frame_q15 = dsp.arm_float_to_q15(frame)

    # Apply the Hann Window. The output is still Q15
    hann_q15 = dsp.arm_mult_q15(frame_q15, hann_lut_q15)

    # Apply the RFFT. The output is Q12.4. Therefore, fewer fractional bits
    fft_spect_q15 = rfft_q15(hann_q15)

    # Calculate the magnitude of the FFT. The output is Q13.3
    fft_mag_spect_q15 = mag_q15(fft_spect_q15)

    # Convert the spectrum to Mel
    log_mel_spect_q15 = dsp.arm_mat_vec_mult_q15(mel_wei_mtx_q15.T, fft_mag_spect_q15.T)

    # Perform the log() function
    for idx, v in enumerate(log_mel_spect_q15):
      log_mel_spect_q15[idx] = log_lut_q13_3[v]

    # Calculate the MFCCs through the DCT
    mfccs = dsp.arm_mat_vec_mult_q15(dct_wei_mtx_q15.T, log_mel_spect_q15)

    # Convert MFCCs to float
    output[i] = mfccs.T / float(8)

  return output

### Extract the MFCC coefficients using the CMSIS-DSP implementation

In [ ]:
mfccs_cmsis = extract_mfccs_cmsis(
    test_ad,
    NUM_MFCCS,
    FRAME_LENGTH,
    FRAME_STEP,
    hann_lut_q15,
    mel_wei_mtx_q15,
    log_lut_q13_3,
    dct_wei_mtx_q15)

### Display MFCCs obtained with the CMSIS-DSP library

In [ ]:
display_mfccs(mfccs_cmsis.T)

### Display the difference

In [ ]:
abs_diff = np.abs(mfccs_tf - mfccs_cmsis)

display_mfccs(abs_diff.T)

print("Differences:\nmin:", np.min(abs_diff),
      "max:", np.max(abs_diff),
      "mean:", np.mean(abs_diff),
      "std:", np.std(abs_diff))

## Designing and training an LSTM model

### Generate the dataset and store it in a JSON file

In [ ]:
import os

x = []
y = []

for genre in LIST_GENRES:
  folder = train_dir + "/" + genre

  list_files = os.listdir(folder)

  for song in list_files:
    filepath = folder + "/" + song

    try:
      ad, sr = sf.read(filepath)

      # Number of splits
      num_it = int(len(ad) / TRAIN_AUDIO_LENGTH_SAMPLES)

      for i in range(num_it):
        s0 = TRAIN_AUDIO_LENGTH_SAMPLES * i
        s1 = s0 + TRAIN_AUDIO_LENGTH_SAMPLES
        src_audio = ad[s0 : s1]

        mfccs = extract_mfccs_cmsis(
            src_audio,
            NUM_MFCCS,
            FRAME_LENGTH,
            FRAME_STEP,
            hann_lut_q15,
            mel_wei_mtx_q15,
            log_lut_q13_3,
            dct_wei_mtx_q15)

        x.append(mfccs.tolist())
        y.append(LIST_GENRES.index(genre))

    except Exception as e:
      continue

### Convert the x and y lists to NumPy arrays

In [ ]:
x, y = np.array(x), np.array(y)

### Split the dataset into train (60%), validation (20%), and test (20%) datasets

In [ ]:
from sklearn.model_selection import train_test_split

# Split 1 (60% vs 40%)
x_train, x0, y_train, y0 = train_test_split(x, y, test_size=0.40, random_state = 1)
# Split 2 (50% vs 50%)
x_test, x_validate, y_test, y_validate = train_test_split(x0, y0, test_size=0.50, random_state = 3)

### Design a many-to-many LSTM model

In [ ]:
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras import layers

input_shape = (x_train.shape[1], x_train.shape[2])

norm_layer = layers.Normalization(axis=-1)

# Learn mean and standard deviation from dataset
norm_layer.adapt(x_train)

input = layers.Input(shape=input_shape)
x = norm_layer(input)
x = layers.LSTM(32, return_sequences=True)(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(len(LIST_GENRES), activation='softmax')(x)

model = tf.keras.Model(input, x)

### Visualize model summary

In [ ]:
model.summary()

### Train the many-to-many LSTM model

In [ ]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

NUM_EPOCHS = 30
BATCH_SIZE = 50

history = model.fit(x_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_validate, y_validate))

### Save the TensorFlow model

In [ ]:
run_model = tf.function(lambda x: model(x))

BATCH_SIZE = 1
STEPS = x_train.shape[1]
FEATURES = x_train.shape[2]

concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, FEATURES], model.inputs[0].dtype))

# model directory.
model.save(TF_MODEL, save_format="tf", signatures=concrete_func)

### Design a many-to-one LSTM model

In [ ]:
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras import layers

input_shape = (x_train.shape[1], x_train.shape[2])

norm_layer = layers.Normalization(axis=-1)

# Learn mean and standard deviation from dataset
norm_layer.adapt(x_train)

input = layers.Input(shape=input_shape)
x = norm_layer(input)
x = layers.LSTM(32, return_sequences=False)(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(len(LIST_GENRES), activation='softmax')(x)

model = tf.keras.Model(input, x)

### Train the many-to-one LSTM model

In [ ]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

NUM_EPOCHS = 30
BATCH_SIZE = 50

history = model.fit(x_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_validate, y_validate))

## Evaluating the quantized TensorFlow Lite model's accuracy on the test dataset

### Select a few hundred of samples randomly from the test dataset to calibrate the quantization

In [ ]:
def representative_data_gen():
  data = tf.data.Dataset.from_tensor_slices(x_test)
  for i_value in data.batch(1).take(100):
    i_value_f32 = tf.dtypes.cast(i_value, tf.float32)
    yield [i_value_f32]

### Quantize the TensorFlow model with the TensorFlow Lite converter

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
tfl_model = converter.convert()

### Initialize the TensorFlow Lite interpreter

In [ ]:
# Initialize the TFLite interpreter
interp = tf.lite.Interpreter(model_content=tfl_model)

### Allocate the tensor and get input quantization parameters

In [ ]:
# Allocate the tensors
interp.allocate_tensors()

# Get input/output layer information
i_details = interp.get_input_details()[0]
o_details = interp.get_output_details()[0]

# Get input quantization parameters.
i_quant = i_details["quantization_parameters"]
i_scale      = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]

### Implement a function to run the model inference using the TensorFlow Lite Python interpreter

In [ ]:
def classify(i_value):
  # Add an extra dimension to the test sample
  # to match the expected 3D tensor shape
  i_value_f32 = np.expand_dims(i_value, axis=0)

  # Quantize (float -> 8-bit) the input
  i_value_f32 = i_value_f32 / i_scale + i_zero_point
  i_value_s8 = tf.cast(i_value_f32, dtype=tf.int8)
  interp.set_tensor(i_details["index"], i_value_s8)

  interp.invoke()

  # TfLite fused Lstm kernel is stateful.
  # Therefore, we need to reset the states before the next inference
  interp.reset_all_variables()

  return interp.get_tensor(o_details["index"])[0]

### Evaluate the accuracy of the quantized TensorFlow Lite model

In [ ]:
num_correct_samples = 0

for i_value, o_value in zip(x_test, y_test):
  o_pred_f32 = classify(i_value)
  o_res = np.argmax(o_pred_f32)
  if o_res == o_value:
    num_correct_samples += 1

num_total_samples   = len(x_test)
print("Accuracy:", num_correct_samples/num_total_samples)

### Convert the TensorFlow model to C-byte array with xxd

In [ ]:
open(TFL_MODEL_FILE, "wb").write(tfl_model)

In [ ]:
!apt-get update && apt-get -qq install xxd
!xxd -i $TFL_MODEL_FILE > model.h
!sed -i 's/unsigned char/const unsigned char/g' model.h

In [ ]:
!cp music_genre_int8.tflite drive/MyDrive/mgr_dataset

### Testing the model with a song

In [ ]:
test_expected_class = 'jazz'
test_song_link = 'https://cdn.pixabay.com/download/audio/2022/05/11/audio_58647a4eda.mp3?filename=jazzy-jazz-110967.mp3'

In [ ]:
test_song_name = 'test_' + test_expected_class + '_song.mp3'

In [ ]:
!wget -O {test_song_name} {test_song_link} .

## Deploying the MFCCs feature extraction algorithm on the Raspberry Pi Pico

### Implement a function to transform a NumPy array to C array

In [ ]:
def to_c_array(data, c_type, filename, num_cols = 12):

  def to_numpy_dt(dtype):
    if dtype == 'float':
      return 'float32'
    if dtype == 'int32_t':
      return 'int32'
    if dtype == 'uint32_t':
      return 'uint32'
    if dtype == 'int16_t':
      return 'int16'
    if dtype == 'uint16_t':
      return 'uint16'
    if dtype == 'int8_t':
      return 'int8'
    if dtype == 'uint8_t':
      return 'uint8'
    return ''

  str_out = ''

  # Write the header guard
  header_guard = filename.upper()
  str_out += '#ifndef ' + header_guard + '\n'
  str_out += '#define ' + header_guard + '\n'

  # Write the tensor dimensions
  # Scan the dimensions in reverse order
  dim_base = 'const int32_t ' + filename + '_dim'
  for idx, dim in enumerate(data.shape[::-1]):
    str_out += dim_base + str(idx) + ' = '
    str_out += str(dim)
    str_out += ';\n'

  # Reshape the NumPy array and cast the array to desired C data type
  np_type  = to_numpy_dt(c_type)
  data_out = data.flatten()
  data_out = data_out.astype(np_type)

  # Write the tensor total size (Optional)
  size = len(data_out)
  sz_base = 'const int32_t ' + filename + '_sz'
  str_out += sz_base + ' = '
  str_out += str(size) + ';\n'

  # Write the array definition
  str_out += 'const ' + c_type + ' ' + filename + '_data[] = '
  str_out += "\n{\n"

  # Write the values
  for i, val in enumerate(data_out):
    str_out += str(val)

    if (i + 1) < len(data_out):
        str_out += ','
    if (i + 1) % num_cols == 0:
        str_out += '\n'

  str_out += '};\n'
  str_out += '#endif\n'

  # Save the C header file
  h_filename = filename + '.h'
  open(h_filename, "w").write(str_out)

### Implement a function to generate a header file with the constants required by the MFCCs feature extraction algorithm

In [ ]:
def to_c_consts(data, filename):
  str_out = ''

  # Write the header guard
  header_guard = filename.upper()
  str_out += '#ifndef ' + header_guard + '\n'
  str_out += '#define ' + header_guard + '\n'

  for x in data:
    value    = x[0]
    var_name = x[1]
    c_type   = x[2]
    str_out += 'const ' + c_type + ' '
    str_out += var_name + ' = '
    str_out += str(value)
    str_out += ';\n'

  str_out += '#endif\n'

  # Save the C header file
  h_filename = filename + '.h'
  open(h_filename, "w").write(str_out)

### Generate the C arrays for all the precomputed components of the MFCCs feature extraction algorithm:

In [ ]:
to_c_array(hann_lut_q15, 'int16_t', 'hann_lut_q15')
to_c_array(mel_wei_mtx_q15.T, 'int16_t', 'mel_wei_mtx_q15_T')
to_c_array(log_lut_q13_3, 'int16_t', 'log_lut_q13_3')
to_c_array(dct_wei_mtx_q15.T, 'int16_t', 'dct_wei_mtx_q15_T')

### Generate the C arrays for the int16 input test and its expected MFCCs in floating-point format

In [ ]:
test_src_q15 = dsp.arm_float_to_q15(test_ad)

to_c_array(test_src_q15, 'int16_t', 'test_src')
to_c_array(mfccs_cmsis, 'float', 'test_dst')

### Generate the C constants required by the MFCCs feature extraction algorithm

In [ ]:
NUM_FRAMES = int(((TRAIN_AUDIO_LENGTH_SAMPLES - FRAME_LENGTH) / FRAME_STEP) + 1)
NUM_FFT_FREQS = int((FFT_LENGTH / 2) + 1)

vars = [
       (FRAME_LENGTH, 'FRAME_LENGTH', 'int32_t'),
       (FRAME_STEP, 'FRAME_STEP', 'int32_t'),
       (NUM_FRAMES, 'NUM_FRAMES', 'int32_t'),
       (FFT_LENGTH, 'FFT_LENGTH', 'int32_t'),
       (NUM_FFT_FREQS, 'NUM_FFT_FREQS', 'int32_t'),
       (NUM_MEL_FREQS, 'NUM_MEL_FREQS', 'int32_t'),
       (NUM_MFCCS, 'NUM_MFCCS', 'int32_t')
       ]

to_c_consts(vars, 'mfccs_consts')